1. В ходе проведения одного из A/B тестов пользователи разбивались 50/50 на две
когорты POS1 и POS0. 

- Суть теста сводится к показу возвратных пушей пользователям, которые сворачивают приложение на сплеш экране, не дожидаясь загрузки рекламы и входа в приложение
- Когорта POS0 пуш не получала, т.е. если пользователь сворачивает приложение на сплеше - ничего не происходит
- Когорта POS1 получала возвратный пуш через 2 секунды после сворачивания приложения на сплеш экране; при клике в пуш пользователь сразу попадал без сплеша и рекламы на главный экран приложения

Было выяснено, что:

a. У когорты POS0 LTV30 было выше, чем у POS1 на 10%, однако среднее
количество показов рекламы на пользователя у POS1 было выше, чем у
POS0 на 5%. 
- С чем может быть связана такая разнонаправленная динамика?
    
b. У когорты POS1 LTV2 на 10% выше, чем у POS0, однако, как было сказано в
п. а). LTV30 затем у POS0 обогнал POS1. 
- Почему это могло произойти?
- Какой можно сделать вывод?
- Какие можно предложить изменения в проведении теста для получения максимального дохода LTV30?

**1. a. Это может указывать на то, что механика возвратных пушей дает большее количество просмотров рекламы, но на долгосрочной перспективе является разждражающим фактором, уменьшает ретеншн и приводит к снижению LTV30**

**1. b. На краткосрочной дистанции вовзратный пуш повышает LTV, но на долгой перспективе раздражает пользователя и приводит к падению LTV. Для потенциального увеличения LTV можно попробовать провести аналогичный тест, но увеличить время, через которое пользователь получит возвратный пуш. Больший период может быть менее раздражителен для пользователя, а так же более информативен, т.к. спустя 2 секунды пользователь, вероятно, еще помнит о приложении, а спустя несколько минут уже может забыть.**

2. На основе данных в BigQuery посчитать, какой % юзеров по США и по Индии и
вообще в среднем по больнице

    a. - смотрят 5 реклам типа интерстишл в первые 7 дней

    b. - смотрят от 10+ интерстишлов в первые 7 дней

    c. - смотрят 15+ интеров в первые 7 дней

    d. - смотрят 20+ интеров в первые 7 дней

    e. и построить на основе этих данных воронку с конверсиями, где шаги
    воронки - количество просмотров интерстишла с шагом 5
- название интересующего нас события - ad_impression_mine - пишется каждый раз, когда пользователю показывается реклама; 
- у события есть дополнительные параметры в event_params
- нас интересует adType = INTERSTITIAL 

In [1]:
import google.cloud.bigquery as bq
import pandas as pd
from itertools import product
from datetime import date
import plotly.express as px
import calendar

In [15]:
client = bq.Client.from_service_account_json("clean-cleaner-dae49-56f2ba8c6441.json")

In [12]:
def dry_run(client: bq.Client, sql: str):
   '''
   Проверка объемов памяти, обрабатываемых запросом
   '''

   job_config = bq.QueryJobConfig(dry_run=True, use_query_cache=False)
   gb_processed = round(client.query(sql, job_config=job_config).total_bytes_processed/(1024 * 1024 * 1024))
   print("Запрос обработает {} Гб".format(gb_processed))

In [20]:
def get_query_interstitial(month_start: int = 1, month_end: int = date.today().month) -> str:
   '''
   Формирование запроса, выводящего воронку просмотров интерстишла для США, Индии и мира
   '''

   base_query = """
               select user_pseudo_id
                    , geo.country
                    , date_diff(timestamp_micros(event_timestamp), timestamp_micros(user_first_touch_timestamp), DAY) as days_since_first_visit
                    , count(*) as cnt_interstitial
                 from `clean-cleaner-dae49.analytics_267738527.events_2023{month}{day}` as t
                    , unnest(event_params) as e
                where event_name = 'ad_impression_mine'
                  and e.key = 'adType' 
                  and e.value.string_value = 'INTERSTITIAL'
                  and date_diff(timestamp_micros(event_timestamp), timestamp_micros(user_first_touch_timestamp), DAY) <= 7
               group by 1, 2, 3
               """
   days = []
   for month in range(month_start, month_end + 1):
      days += [day_of_month for day_of_month in list(product([str(month).zfill(2)],
                                                             [str(day).zfill(2) for day in sum(calendar.Calendar().monthdayscalendar(date.today().year, month), []) \
                                                                if day > 0 
                                                                and ((month == date.today().month and day < date.today().day - 1) \
                                                                or (month < date.today().month))]))]

   union_query = """"""
   for month, day in days:
      union_query += '\n union all\n' + base_query.format(month=month, day=day)
   union_query = union_query.lstrip('\n union all\n')

   complete_query = """
               with union_query as ({union_query})

               , usa_or_india as (
                  select country
                     , 5*floor(cnt_interstitial/5) as funnel_level
                     , count(user_pseudo_id) as cnt_users
                     from (select country
                              , user_pseudo_id
                              , sum(cnt_interstitial) as cnt_interstitial
                           from union_query
                           where country in ('United States', 'India')
                           group by 1, 2)
                  group by 1, 2)

               , world as (
                  select funnel_level
                       , avg(cnt_users) as cnt_users
                    from (select country
                               , 5*floor(cnt_interstitial/5) as funnel_level
                               , count(user_pseudo_id) as cnt_users
                            from (select country
                                     , user_pseudo_id
                                     , sum(cnt_interstitial) as cnt_interstitial
                                  from union_query
                                  group by 1, 2)
                          group by 1, 2)
                  group by 1)
                                    

               , world as (
                  select 5*floor(cnt_interstitial/5) as funnel_level
                     , count(user_pseudo_id) as cnt_users
                     from (select user_pseudo_id
                              , sum(cnt_interstitial) as cnt_interstitial
                           from union_query
                           group by 1)
                  group by 1)    

               select country, funnel_level, cnt_users
               from (select country, funnel_level, cnt_users from usa_or_india union all
                     select 'World' as country, funnel_level, cnt_users from world)
               """.format(union_query=union_query)

   return complete_query

def show_funnel_interstitial(df: pd.DataFrame):
   '''
   Отображение воронки по количеству просмотров интерстишла
   '''

   df = df.rename(columns={'country': 'Локация', 'cnt_users': 'Кол-во пользователей', 'funnel_level': 'Уровень воронки'})
   fig0 = px.funnel(df,
                  x='Кол-во пользователей',
                  y='Уровень воронки',
                  color='Локация',
                  title='Воронка просмотров INTERSTITIAL', opacity=0.9)
   fig0.show()

In [5]:
interstitial_query = get_query_interstitial(month_start=1, month_end=2)

In [13]:
dry_run(client, interstitial_query)

Запрос обработает 582 Гб


In [9]:
# df = client.query(interstitial_query).to_dataframe()

In [22]:
df = pd.DataFrame([['India', '5', 111171], ['India', '10', 252525], ['India', '20', 323312], ['India', '25', 256312], ['India', '30', 206312],
                   ['United States', '5', 89171], ['United States', '10', 202525], ['United States', '20', 223312], ['United States', '25', 183312], ['United States', '30', 153312],
                   ['World', '5', 78171], ['World', '10', 178525], ['World', '20', 203312], ['World', '25', 163312], ['World', '30', 133565],
                   ], columns=['country', 'funnel_level', 'cnt_users'])

In [23]:
show_funnel_interstitial(df)

3. На основе данных в BigQuery необходимо сделать аналитику по проводимому A/B
тесту, в котором есть 3 когорты пользователей (все они атрибутированы внутри BQ
в рамках каждого события) и сделать по ним вывод следующей информации

    a. LTV1/7/14

    b. Ретеншн 1-7-14 дня по событию просмотра рекламы

    c. Кол-во чисток на пользователя

    d. % пользователей, не сделавших ни одной чистки

    e. % пользователей, не посмотревших ни одной рекламы

    f. кол-во показов рекламы на пользователя в разрезе рекламных блоков
- Тест называется TOSI по названию переменной, которая бьет пользователей на 3 когорты
    - TOSI=0 - baseline, нет таймаута на сплеш экране
    - TOSI=1 - таймаут 10 секунд, если за это время не загружается реклама - пропускаем пользователя дальше на главный экран
    - TOSI=2 - таймаут 15 секунд
- Параметр tosi можно найти внутри traffic_source.name (атрибуция идет на 100% ивентов пока лился трафик), вид там обычно такой - “nikitak|tavAdsterra|pop|tosi1”, где “|tosiX” находится в 4-й позиции в строке
- Трафик лился с 09.01.2023 по 30.01.2023
- Для подсчета LTV нужно использовать событие paid_ad_impression, внутри event_params есть value & valueMicros, value - значение в долларах, valueMicros - в микродолларах
- Для подсчета ретеншна надо использовать событие ad_impression_mine (мы считаем пользователя вернувшимся, если он посмотрел хотя бы одну рекламу)
- Для подсчета количества чисток на пользователя нужно использовать события (1 событие - 1 чистка)
    - action_cpu
    - action_memory
    - action_energy
    - action_junk
    - memory_cleaning_start

In [ ]:
def get_query_tosi() -> str:
   '''
   Формирование запроса, выводящего аналитические показатели по тесту TOSI
   '''

   base_query_ltv = """
                    select user_pseudo_id
                         , geo.country
                         , date_diff(timestamp_micros(event_timestamp), timestamp_micros(user_first_touch_timestamp), DAY) as days_since_first_visit
                         , count(*) as cnt_interstitial
                      from `clean-cleaner-dae49.analytics_267738527.events_2023{month}{day}` as t
                         , unnest(event_params) as e
                     where event_name = 'paid_ad_impression'
                       and traffic_source.name like '%|%|%|tosi%'
                       and e.
                    group by 1, 2, 3
                    """

   base_query_retention = """
                            select user_pseudo_id
                                 , geo.country
                                 , date_diff(timestamp_micros(event_timestamp), timestamp_micros(user_first_touch_timestamp), DAY) as days_since_first_visit
                                 , count(*) as cnt_interstitial
                            from `clean-cleaner-dae49.analytics_267738527.events_2023{month}{day}` as t
                                , unnest(event_params) as e
                            where event_name = 'ad_impression_mine'
                                and traffic_source.name like '%|%|%|tosi%'
                            group by 1, 2, 3
                            """

   base_query_cleaning = """
                            select user_pseudo_id
                                 , geo.country
                                 , date_diff(timestamp_micros(event_timestamp), timestamp_micros(user_first_touch_timestamp), DAY) as days_since_first_visit
                                 , count(*) as cnt_interstitial
                            from `clean-cleaner-dae49.analytics_267738527.events_2023{month}{day}` as t
                                , unnest(event_params) as e
                            where event_name in ('action_cpu', 'action_memory', 'action_energy', 'action_junk', 'memory_cleaning_start')
                                and traffic_source.name like '%|%|%|tosi%'
                            group by 1, 2, 3
                            """

   days = [day_of_month for day_of_month in list(product(['01'], [str(day).zfill(2) for day in range(9, 31)]))]

   union_query = """"""
   for month, day in days:
      union_query += '\n union all\n' + base_query.format(month=month, day=day)
   union_query = union_query.lstrip('\n union all\n')

   complete_query = """
               with union_query as ({union_query})

               """.format(union_query=union_query)

   return complete_query

def show_metrics_tosi(df: pd.DataFrame):
   '''
   Отображение метрик по тесту TOSI
   '''

   pass

4. Посмотреть на наше приложение и события аналитические за декабрь-январь и
попробовать на основе этих данных

    a. найти 2 “инсайта” (например, что юзеры на таком-то телефоне или такой-то
    версии несут больше LTV, чем другие)

    b. визуализировать воронки входа в наше приложение (от запуска до
    попадания на главный экран) в зависимости от одного из 4 сценариев (вход
    через иконку, вход через серверный пуш, вход через триггерный пуш, вход
    через виджет), где начало воронки - клик в иконку или пуш, а конец воронки
    - открытие главной страницы (по пути есть события, все они будут
    переданы)

        i. т.е. на выходе ожидаем для юзеров, установивших приложение с 1 по
        31 декабря увидеть их на горизонте от 1 до 30 января как они
        попадают в приложение в % от общего количества пользователей
        (например, на первый день 50% входят через виджет и 10% через
        пуш, а на 30-й день 10% через виджет и 40% через серверный пуш)

            1. если юзер входил в приложение в один день разными путями -
            мы считаем его несколько раз, таким образом, юзеров может
            быть более 100% по дню
Задача является ОПЦИОНАЛЬНОЙ! (не является обязательной и скорее является “бонус треком”, т.к. есть понимание, что она может быть бесконечно сложной)
- Для решения задачи нас интересует целая группа событий с префиксом ias_*
- Нас не интересуют пользователи, у которых нет ни одного события ias_hide_splash и у которых было в атрибуции tosi=1 или tosi=2 (из задачи 3)
- В каждом из этих событий есть в параметрах поле scenario, которое принимает следующие интересующие нас значения:
    - 1- вход через иконку
    - 10 - вход через серверный пуш
    - 6 - вход через триггерный пуш
    - 4 - вход через виджет
- Воронка событий выглядит примерно так (вначале не пишу ias_*, все события начинаются с этих волшебных 4 символов)
    - show_splash_on_create - вход на сплеш
    - load_started - старт прогрузки рекламы Interstitial after splash
    - showed или show_failed - можно считать концом этой воронки (реклама либо показана, либо нет)
- Дополнительно хотелось бы узнать, у какого % пользователей от всего количества не было ни разу показа рекламы после сплеша на основе следующих событий
    - load_failed_final - финальный фэил загрузки, больше попыток не будет
    - show_failed - ошибка показа рекламы после сплеша
    - showed - реклама успешно показана после сплеша